## Eawag Readers Workspace

In [132]:
# imports
from datetime import datetime
import pandas as pd
import xarray as xr
import pyrsktools as rsk

from mooring_reader import MooringReader

In [2]:
lake = 'Zugersee'
location = 'LZ1'
year = '2025'
date = '20250424'
bathy_file = '../../../data/lake_zug/bathymetry/zugersee_bathy.nc'

reader = MooringReader(lake, location, year, date, bathy_file)

In [3]:
reader.__dict__

{'lake': 'Zugersee',
 'location': 'LZ1',
 'year': '2025',
 'date': '20250424',
 'bathy_file': '../../../data/lake_zug/bathymetry/zugersee_bathy.nc',
 'datalakes': False,
 'md_file': 'Q:/Messdaten/Aphys_Hypothesis_data/Zugersee/2025/Mooring/20250424/LZ1_md.json'}

### Load oxygen logger data into xarray Dataset

In [30]:
import dolfyn as dlfn

In [31]:
ds = dlfn.read(f'Q:/Messdaten/Aphys_Hypothesis_data/{lake}/{year}/Mooring/{date}/{location}/6D728000.000')


Reading file Q:/Messdaten/Aphys_Hypothesis_data/Zugersee/2025/Mooring/20250424/LZ1/6D728000.000 ...


In [32]:
ds

<xarray.Dataset> Size: 20MB
Dimensions:              (time: 14243, dir: 4, range: 45, beam: 4, x: 4, x*: 4,
                          earth: 3, inst: 3)
Coordinates:
  * time                 (time) datetime64[ns] 114kB 2025-01-15T12:12:27.0224...
  * dir                  (dir) <U3 48B 'E' 'N' 'U' 'err'
  * range                (range) float64 360B 2.08 3.08 4.08 ... 45.08 46.08
  * beam                 (beam) int64 32B 1 2 3 4
  * x                    (x) int64 32B 1 2 3 4
  * x*                   (x*) int64 32B 1 2 3 4
  * earth                (earth) <U1 12B 'E' 'N' 'U'
  * inst                 (inst) <U1 12B 'X' 'Y' 'Z'
Data variables: (12/21)
    number               (time) uint32 57kB 1 2 3 4 ... 14240 14241 14242 14243
    builtin_test_fail    (time) bool 14kB False False False ... False False
    c_sound              (time) float32 57kB 1.476e+03 1.474e+03 ... 1.449e+03
    depth                (time) float32 57kB 150.0 150.0 150.0 ... 150.0 150.0
    pitch                (time) float32 57kB 24.88 24.73 24.73 ... 1.92 3.69
    roll                 (time) float32 57kB 14.58 7.73 7.55 ... -25.94 -25.95
    ...                   ...
    vel                  (dir, range, time) float32 10MB nan nan nan ... nan nan
    amp                  (beam, range, time) uint8 3MB 41 41 41 41 ... 42 42 42
    corr                 (beam, range, time) uint8 3MB 17 16 13 13 ... 75 11 0 0
    prcnt_gd             (beam, range, time) uint8 3MB 0 0 0 0 0 ... 25 0 0 0 0
    beam2inst_orientmat  (x, x*) float64 128B 1.462 -1.462 0.0 ... -1.034 -1.034
    orientmat            (earth, inst, time) float64 1MB 0.9562 ... -0.8977
Attributes: (12/39)
    inst_make:                TRDI
    inst_type:                ADCP
    rotate_vars:              ['vel']
    has_imu:                  0
    prog_ver:                 51.42
    inst_model:               Workhorse
    ...                       ...
    transmit_lag_m:           0.23
    bandwidth:                0
    power_level:              255
    serialnum:                1728
    fs:                       0.007142857142857143
    vel_gps_corrected:        0

In [110]:
serial_id = '893460'
sensor = 'minidot'
fpath = f'Q:/Messdaten/Aphys_Hypothesis_data/{lake}/{year}/Mooring/{date}/{location}/7450-{serial_id}/Cat.txt'
fpath

'Q:/Messdaten/Aphys_Hypothesis_data/Zugersee/2025/Mooring/20250424/LZ1/7450-893460/Cat.txt'

In [118]:
def load_from_L0(fpath, sensor, depth, serial_id):
    cols_drop = ['Unix Timestamp', 'Coordinated Universal Time', 'Battery', 'Q']

    cols_map = {
        'UTC_Date_&_Time': 'time',
        'Temperature': 'temp',
        'Dissolved Oxygen': 'd_oxygen',
        'Dissolved Oxygen Saturation': 'd_oxygen_sat'
    }
    
    if sensor == 'minidot':
        with open(fpath, 'r') as f:
            lines = [x[:-1] for x in f if len(x.split(',')) > 1]

        # extract column names and units
        cols = [x.lstrip(' ') for x in lines[0].split(',')]

        data = []
        for line in lines[2:]:
            data.append([x.lstrip(' ') for x in line.split(',')])
        data = pd.DataFrame(data, columns=cols)

        data = data.drop(cols_drop, axis=1)
        data = data.rename(columns=cols_map)
        data['time'] = pd.to_datetime(data['time'])
        data['temp'] = data['temp'].astype(float)
        data['d_oxygen'] = data['d_oxygen'].astype(float)
        data['d_oxygen_sat'] = data['d_oxygen_sat'].astype(float)
        data = data.set_index('time')

        ds = xr.Dataset.from_dataframe(data)
        ds = ds.assign_coords(depth=depth, serial_id=serial_id)

    return ds

In [112]:
ds = load_from_L0(fpath, sensor, 100, serial_id)
ds

<xarray.Dataset> Size: 5MB
Dimensions:       (time: 151233)
Coordinates:
  * time          (time) datetime64[ns] 1MB 2025-01-09T14:59:00 ... 2025-04-2...
    serial_id     <U6 24B '893460'
    depth         int64 8B 100
Data variables:
    temp          (time) float64 1MB 18.09 18.18 18.18 18.2 ... 21.62 21.62 21.6
    d_oxygen      (time) float64 1MB 8.668 8.636 8.647 ... 8.294 8.281 8.277
    d_oxygen_sat  (time) float64 1MB 96.46 96.27 96.4 ... 99.04 98.89 98.81

In [113]:
serial_id = '899457'
sensor = 'minidot'
fpath = f'Q:/Messdaten/Aphys_Hypothesis_data/{lake}/{year}/Mooring/{date}/{location}/7450-{serial_id}/Cat.txt'
fpath

'Q:/Messdaten/Aphys_Hypothesis_data/Zugersee/2025/Mooring/20250424/LZ1/7450-899457/Cat.txt'

In [114]:
ds2 = load_from_L0(fpath, sensor, 200, serial_id)
ds2

<xarray.Dataset> Size: 5MB
Dimensions:       (time: 151227)
Coordinates:
  * time          (time) datetime64[ns] 1MB 2025-01-09T14:49:00 ... 2025-04-2...
    serial_id     <U6 24B '899457'
    depth         int64 8B 200
Data variables:
    temp          (time) float64 1MB 18.59 18.77 18.82 ... 21.98 22.02 22.0
    d_oxygen      (time) float64 1MB 8.501 8.461 8.455 ... 8.153 8.144 8.147
    d_oxygen_sat  (time) float64 1MB 95.57 95.47 95.5 95.6 ... 98.03 98.02 98.01

In [115]:
ds_aligned = xr.align(*[ds, ds2], join='inner')

In [116]:
ds_combined = xr.concat(ds_aligned, dim='depth')

C:\Users\herremha\AppData\Local\Temp\ipykernel_30012\684412241.py:1: FutureWarning: In a future version of xarray the default value for coords will change from coords='different' to coords='minimal'. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set coords explicitly.
  ds_combined = xr.concat(ds_aligned, dim='depth')


In [117]:
ds_combined

<xarray.Dataset> Size: 8MB
Dimensions:       (depth: 2, time: 151217)
Coordinates:
  * time          (time) datetime64[ns] 1MB 2025-01-09T14:59:00 ... 2025-04-2...
    serial_id     (depth) <U6 48B '893460' '899457'
  * depth         (depth) int64 16B 100 200
Data variables:
    temp          (depth, time) float64 2MB 18.09 18.18 18.18 ... 22.02 22.0
    d_oxygen      (depth, time) float64 2MB 8.668 8.636 8.647 ... 8.144 8.147
    d_oxygen_sat  (depth, time) float64 2MB 96.46 96.27 96.4 ... 98.02 98.01

### Load thermistor data into xarray Dataset

In [130]:
serial_id = '200569'
sensor = 'rbr_temp'
fpath = f'Q:/Messdaten/Aphys_Hypothesis_data/{lake}/{year}/Mooring/{date}/{location}/AUTO_{serial_id}_{date}_1356.rsk'
fpath

'Q:/Messdaten/Aphys_Hypothesis_data/Zugersee/2025/Mooring/20250424/LZ1/AUTO_200569_20250424_1356.rsk'

In [147]:
with rsk.RSK(fpath) as f:
    f.readdata()
    serial_id = str(f.instrument.serialID)
    data = pd.DataFrame(f.data)

In [148]:
cols_map = {
    'timestamp': 'time',
    'temperature': 'temp'
}

data = data.rename(columns=cols_map)
data = data.set_index('time')
data

,temp
time,
2025-01-15 06:00:00,16.661611
2025-01-15 06:00:10,16.657880
2025-01-15 06:00:20,16.657910
2025-01-15 06:00:30,16.657086
2025-01-15 06:00:40,16.643454
...,...
2025-04-24 13:55:40,21.118636
2025-04-24 13:55:50,21.115041
2025-04-24 13:56:00,21.073992


In [149]:
xr.Dataset.from_dataframe(data)

<xarray.Dataset> Size: 14MB
Dimensions:  (time: 858219)
Coordinates:
  * time     (time) datetime64[ms] 7MB 2025-01-15T06:00:00 ... 2025-04-24T13:...
Data variables:
    temp     (time) float64 7MB 16.66 16.66 16.66 16.66 ... 21.07 21.11 21.27